In [1]:
from datetime import datetime, timedelta
import math 

import DIN5034_2
import EN17037_sunPos

In [2]:
place = "Innsbruck"
country = "AUT"
lat = 47.27
lon = 11.35
mer = 15
elev = 584.0

turbidity = [2.5, 3.5, 4.5, 5.5]

# Create a datetime object for the beginning of the year
start_of_year = datetime(2025, 1, 1)

In [9]:
with open("innsbruck_din5034_overcast.wea", "w") as f:
    f.write("place " + place + "_" + country + "\n")
    f.write("latitude " + str(lat) + "\n")
    f.write("longitude " + str((-1)*lon) + "\n") 
    f.write("time_zone " + str((-1)*mer) + "\n")
    f.write("site_elevation " + str(elev) + "\n")
    f.write("weather_data_file_units 1" + "\n")

    for day_idx in range(365):
        for hr_idx in range(96):
            
            hr = hr_idx / 4 + 0.125
            day = day_idx + 1
            # Add the timedelta corresponding to the day of year and hour
            target_datetime = start_of_year + timedelta(days=day_idx, hours=hr)
    
            # Format the datetime object into the desired string format
            formatted_date = target_datetime.strftime("%#m %#d")
    
            # Get sun positions
            pos = EN17037_sunPos.sunPos(hr, lat, lon, mer, day)
            gamma_s = math.radians(pos[0])
            alpha_s = math.radians(pos[1])
    
            # Get horizontal irradiance under overcast sky
            if gamma_s < 0.0: 
                Ee = 0.0 
            else:
                Ee = DIN5034_2.E_e(gamma_s)

            f.write(f"{formatted_date} {hr} 0 {Ee:.0f}\n")


In [5]:
for turb in turbidity: 

    with open("innsbruck_din5034_clear_t" + str(turb) + ".wea", "w") as f:
        f.write("place " + place + "_" + country + "\n")
        f.write("latitude " + str(lat) + "\n")
        f.write("longitude " + str((-1)*lon) + "\n") 
        f.write("time_zone " + str((-1)*mer) + "\n")
        f.write("site_elevation " + str(elev) + "\n")
        f.write("weather_data_file_units 1" + "\n")

        for day_idx in range(365):
            for hr_idx in range(96):
                
                hr = hr_idx / 4 + 0.125
                day = day_idx + 1
                # Add the timedelta corresponding to the day of year and hour
                target_datetime = start_of_year + timedelta(days=day_idx, hours=hr)
        
                # Format the datetime object into the desired string format
                formatted_date = target_datetime.strftime("%#m %#d")
        
                # Get sun positions
                pos = EN17037_sunPos.sunPos(hr, lat, lon, mer, day)
                gamma_s = math.radians(pos[0])
                alpha_s = math.radians(pos[1])
        
                if gamma_s < 0.0: 
                    Ee = 0.0 
                    EeH = 0.0
                    EeS = 0.0
                    EeS_dir = 0.0
                else:
        
                    # Get horizontal irradiance under sunny sky for sky:
                    EeH = DIN5034_2.E_eH(gamma_s, turb, elev)
    
                    # Get horizontal irradiance under sunny sky for sun:
                    EeS = DIN5034_2.E_eS(gamma_s, turb, elev)

                    # Calculate direct normal irradiance
                    EeS_dir = EeS / math.sin(gamma_s)

                f.write(f"{formatted_date} {hr} {EeS_dir:.0f} {EeH:.0f}\n")